In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from gensim.models import Word2Vec
import ast # CSV'den dize literal gösterimlerini güvenli bir şekilde değerlendirmek için
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer, PorterStemmer

# NLTK verilerini indirme (henüz indirilmediyse)
try:
    stopwords.words('turkish') # Türkçe stop kelimeleri kontrol ediyorum
except LookupError:
    nltk.download('stopwords')
try:
    nltk.data.find('tokenizers/punkt')
except LookupError:
    nltk.download('punkt')
try:
    nltk.data.find('corpora/wordnet')
except LookupError:
    nltk.download('wordnet')

print("Gerekli kütüphaneler ve NLTK verileri yüklendi.")

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\yunus\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


Gerekli kütüphaneler ve NLTK verileri yüklendi.


[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\yunus\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [7]:
pip install gensim

Note: you may need to restart the kernel to use updated packages.


In [2]:
# Verilerimin ve modellerimin bulunduğu yolları ayarlıyorum
DATA_DIR = "C:/Users/yunus/Desktop/Bahar Dersleri/Yapay Zeka Proje/arac-arizadeseni-eslestirme/data/processed/"
MODELS_DIR = "C:/Users/yunus/Desktop/Bahar Dersleri/Yapay Zeka Proje/arac-arizadeseni-eslestirme/models/"
RAW_DATA_DIR = "C:/Users/yunus/Desktop/Bahar Dersleri/Yapay Zeka Proje/arac-arizadeseni-eslestirme/data/raw/"


LEM_DATA_PATH = DATA_DIR + "preprocessed_data_lemmatized_only.csv"
STEM_DATA_PATH = DATA_DIR + "preprocessed_data_stemmed_only.csv"
ALL_RAW_DATA_PATH = RAW_DATA_DIR + "all_data.csv" # Orijinal yorumları görüntülemek için


# Word2Vec model parametreleri (eğittiğim şekilde eşleşmeli)
W2V_PARAMS = [
    {'model_type': 'cbow', 'window': 2, 'vector_size': 100},
    {'model_type': 'skipgram', 'window': 2, 'vector_size': 100},
    {'model_type': 'cbow', 'window': 4, 'vector_size': 100},
    {'model_type': 'skipgram', 'window': 4, 'vector_size': 100},
    {'model_type': 'cbow', 'window': 2, 'vector_size': 300},
    {'model_type': 'skipgram', 'window': 2, 'vector_size': 300},
    {'model_type': 'cbow', 'window': 4, 'vector_size': 300},
    {'model_type': 'skipgram', 'window': 4, 'vector_size': 300}
]

print("Konfigürasyon tamamlandı.")

Konfigürasyon tamamlandı.


In [3]:
lemmatizer = WordNetLemmatizer()
stemmer = PorterStemmer()

def preprocess_text_lemmatized(text):
    sentences = nltk.sent_tokenize(text)
    processed_sentences = []
    for sentence in sentences:
        sentence = sentence.lower()
        sentence = re.sub(r'[^\\w\\s]', '', sentence)
        tokens = nltk.word_tokenize(sentence)
        stop_words = set(stopwords.words('english')) # Verilerim İngilizce göründüğü için 'english' kullanıyorum
        tokens = [token for token in tokens if token not in stop_words and token.isalpha()]
        tokens = [lemmatizer.lemmatize(token) for token in tokens]
        processed_sentences.append(' '.join(tokens)) # Belirteçleri TF-IDF için tekrar dizeye birleştiriyorum
    return ' '.join(processed_sentences) # TF-IDF için tek bir dize olarak döndürüyorum

def preprocess_text_stemmed(text):
    sentences = nltk.sent_tokenize(text)
    processed_sentences = []
    for sentence in sentences:
        sentence = sentence.lower()
        sentence = re.sub(r'[^\\w\\s]', '', sentence)
        tokens = nltk.word_tokenize(sentence)
        stop_words = set(stopwords.words('english')) # Verilerim İngilizce göründüğü için 'english' kullanıyorum
        tokens = [token for token in tokens if token not in stop_words and token.isalpha()]
        tokens = [stemmer.stem(token) for token in tokens]
        processed_sentences.append(' '.join(tokens)) # Belirteçleri TF-IDF için tekrar dizeye birleştiriyorum
    return ' '.join(processed_sentences) # TF-IDF için tek bir dize olarak döndürüyorum

# Word2Vec için belirteç listesi gerekiyor, tek bir dize değil
def preprocess_text_w2v(text, preprocessor_type='lemmatized'):
    sentences = nltk.sent_tokenize(text)
    all_tokens = []
    for sentence in sentences:
        sentence = sentence.lower()
        sentence = re.sub(r'[^\\w\\s]', '', sentence)
        tokens = nltk.word_tokenize(sentence)
        stop_words = set(stopwords.words('english'))
        tokens = [token for token in tokens if token not in stop_words and token.isalpha()]
        if preprocessor_type == 'lemmatized':
            tokens = [lemmatizer.lemmatize(token) for token in tokens]
        elif preprocessor_type == 'stemmed':
            tokens = [stemmer.stem(token) for token in tokens]
        all_tokens.extend(tokens) # Tüm belge için düz bir belirteç listesi elde etmek için append yerine extend kullanıyorum
    return all_tokens

# Yardımcı fonksiyon: Orijinal yorumları alma
def get_original_comment(doc_id):
    """Bir document_id verildiğinde orijinal yorum metnini alır."""
    # doc_id 'doc1' gibi, bunu bir tamsayı dizinine dönüştürmem gerekiyor
    idx = int(doc_id.replace('doc', '')) - 1
    if idx < len(df_raw):
        return df_raw.loc[idx, 'comments']
    return "Yorum bulunamadı."

# Önceden işlenmiş verileri yüklüyorum
df_raw = pd.read_csv(ALL_RAW_DATA_PATH)
df_raw['document_id'] = ['doc' + str(i + 1) for i in df_raw.index]

df_lemmatized_processed = pd.read_csv(LEM_DATA_PATH)
df_stemmed_processed = pd.read_csv(STEM_DATA_PATH)

# Listelerin dize gösterimlerini gerçek belirteç listelerine geri dönüştürüyorum
df_lemmatized_processed['comments_processed'] = df_lemmatized_processed['comments_processed'].apply(ast.literal_eval)
df_stemmed_processed['comments_stemmed'] = df_stemmed_processed['comments_stemmed'].apply(ast.literal_eval)

# Giriş metnini tanımlıyorum ve ön işleme tabi tutuyorum
input_text_id = 'doc1' # PDF yönergelerine göre doc1'i seçtim [cite: 92]
input_text_raw = df_raw[df_raw['document_id'] == input_text_id]['comments'].iloc[0]
print(f"--- Giriş Metni (Orijinal): {input_text_id} ---\n'{input_text_raw}'\n")

input_text_lemmatized_tfidf = preprocess_text_lemmatized(input_text_raw)
input_text_stemmed_tfidf = preprocess_text_stemmed(input_text_raw)
input_tokens_lemmatized_w2v = preprocess_text_w2v(input_text_raw, 'lemmatized')
input_tokens_stemmed_w2v = preprocess_text_w2v(input_text_raw, 'stemmed')

print("Veriler yüklendi ve ön işleme fonksiyonları hazır.")

--- Giriş Metni (Orijinal): doc1 ---
'Since 2011, there have been numerous issues. The vehicle has not reached 40,000 miles because there is always something wrong with it. As of now, there is transmission failure, jerking, shuddering, electrical problems, and steering problems. Also, the engine dies while driving. I want to get rid of it, but CarMax told the vehicle is worthless in its condition. This car has been a nightmare.'

Veriler yüklendi ve ön işleme fonksiyonları hazır.


In [8]:
print("\n--- TF-IDF Benzerlik Hesaplamaları ---")

# TF-IDF Lemmatized
print("\nTF-IDF (Lemmatized) Modeli:")
# Tutarlı bir kelime hazinesi ve IDF sağlamak için TfidfVectorizer'ı tüm metin üzerinde yeniden eğitiyorum.
# TF-IDF vektörleştirme için tüm yorumları tek bir dize listesinde birleştiriyorum.
corpus_lemmatized_tfidf = [ ' '.join(tokens) for sublist in df_lemmatized_processed['comments_processed'] for tokens in sublist ]
tfidf_vectorizer_lemma = TfidfVectorizer()
tfidf_matrix_lemma = tfidf_vectorizer_lemma.fit_transform(corpus_lemmatized_tfidf)
input_vector_lemma = tfidf_vectorizer_lemma.transform([input_text_lemmatized_tfidf])
cosine_similarities_lemma = cosine_similarity(input_vector_lemma, tfidf_matrix_lemma).flatten()

# En benzer 5 belgeyi alıyorum
top_5_indices_lemma_tfidf = cosine_similarities_lemma.argsort()[-5:][::-1]
top_5_docs_lemma_tfidf = []
for i in top_5_indices_lemma_tfidf:
    doc_id = 'doc' + str(i + 1)
    similarity_score = cosine_similarities_lemma[i]
    top_5_docs_lemma_tfidf.append({'document_id': doc_id, 'score': similarity_score, 'text': get_original_comment(doc_id)})

print("En Benzer 5 Belge (TF-IDF Lemmatized):")
for doc in top_5_docs_lemma_tfidf:
    print(f"  Belge ID: {doc['document_id']}, Skor: {doc['score']:.4f}, Metin: '{doc['text']}'")

# TF-IDF Stemmed
print("\nTF-IDF (Stemmed) Modeli:")
corpus_stemmed_tfidf = [ ' '.join(tokens) for sublist in df_stemmed_processed['comments_stemmed'] for tokens in sublist ]
tfidf_vectorizer_stem = TfidfVectorizer()
tfidf_matrix_stem = tfidf_vectorizer_stem.fit_transform(corpus_stemmed_tfidf)
input_vector_stem = tfidf_vectorizer_stem.transform([input_text_stemmed_tfidf])
cosine_similarities_stem = cosine_similarity(input_vector_stem, tfidf_matrix_stem).flatten()

# En benzer 5 belgeyi alıyorum
top_5_indices_stem_tfidf = cosine_similarities_stem.argsort()[-5:][::-1]
top_5_docs_stem_tfidf = []
for i in top_5_indices_stem_tfidf:
    doc_id = 'doc' + str(i + 1)
    similarity_score = cosine_similarities_stem[i]
    top_5_docs_stem_tfidf.append({'document_id': doc_id, 'score': similarity_score, 'text': get_original_comment(doc_id)})

print("En Benzer 5 Belge (TF-IDF Stemmed):")
for doc in top_5_docs_stem_tfidf:
    print(f"  Belge ID: {doc['document_id']}, Skor: {doc['score']:.4f}, Metin: '{doc['text']}'")

print("TF-IDF benzerlik hesaplamaları tamamlandı.")


--- TF-IDF Benzerlik Hesaplamaları ---

TF-IDF (Lemmatized) Modeli:
En Benzer 5 Belge (TF-IDF Lemmatized):
  Belge ID: doc2888, Skor: 0.0000, Metin: 'Yorum bulunamadı.'
  Belge ID: doc970, Skor: 0.0000, Metin: 'Yorum bulunamadı.'
  Belge ID: doc968, Skor: 0.0000, Metin: 'Yorum bulunamadı.'
  Belge ID: doc967, Skor: 0.0000, Metin: 'Yorum bulunamadı.'
  Belge ID: doc966, Skor: 0.0000, Metin: 'Yorum bulunamadı.'

TF-IDF (Stemmed) Modeli:
En Benzer 5 Belge (TF-IDF Stemmed):
  Belge ID: doc429, Skor: 0.3396, Metin: 'Engine burnt oil rather quickly. I have to put in at least 3 Qt. of 5W-30 every week (~ 300 Miles - freeway driving).'
  Belge ID: doc971, Skor: 0.0000, Metin: 'Yorum bulunamadı.'
  Belge ID: doc969, Skor: 0.0000, Metin: 'Yorum bulunamadı.'
  Belge ID: doc968, Skor: 0.0000, Metin: 'Yorum bulunamadı.'
  Belge ID: doc967, Skor: 0.0000, Metin: 'Yorum bulunamadı.'
TF-IDF benzerlik hesaplamaları tamamlandı.


In [5]:
print("\n--- Word2Vec Benzerlik Hesaplamaları ---")

# Ortalama vektör hesaplama fonksiyonu (Word2Vec için)
def get_avg_vector(tokens, model):
    vectors = []
    for word in tokens:
        if word in model.wv:
            vectors.append(model.wv[word])
    if vectors:
        return np.mean(vectors, axis=0)
    # Modelde vektör temsili olmayan kelime mevcutsa bunları atlamak aksi taktirde hata mesajı alırım
    return np.zeros(model.vector_size) # Modelde kelime bulunamazsa sıfır vektörü döndürüyorum

# Jaccard karşılaştırması için tüm sonuçları saklıyorum
all_model_top_docs = {}

# Lemmatized Word2Vec modelleri için işlem yapıyorum
for param in W2V_PARAMS:
    model_type = param['model_type']
    vector_size = param['vector_size']
    window = param['window']
    model_filename = f"lemmatized_model_{model_type}_vs{vector_size}_w{window}.model"
    model_name = f"Lemmatized Word2Vec ({model_type}, VS={vector_size}, W={window})"

    try:
        model = Word2Vec.load(MODELS_DIR + model_filename)
    except FileNotFoundError:
        print(f"Uyarı: {model_filename} modeli bulunamadı. Atlanıyor.")
        continue

    print(f"\n{model_name}:")

    # Giriş metni için ortalama vektörü alıyorum
    input_avg_vec = get_avg_vector(input_tokens_lemmatized_w2v, model)

    # Korpustaki tüm belgelerle benzerlikleri hesaplıyorum
    document_vectors = []
    for doc_token_lists in df_lemmatized_processed['comments_processed']:
        flat_doc_tokens = [token for sublist in doc_token_lists for token in sublist]
        document_vectors.append(get_avg_vector(flat_doc_tokens, model))

    document_vectors_matrix = np.array(document_vectors)

    # Giriş vektörü ile tüm belge vektörleri arasındaki kosinüs benzerliğini hesaplıyorum
    similarities = cosine_similarity(input_avg_vec.reshape(1, -1), document_vectors_matrix).flatten()

    # En benzer 5 belgeyi alıyorum
    top_5_indices = similarities.argsort()[-5:][::-1]
    top_5_docs = []
    for i in top_5_indices:
        doc_id = 'doc' + str(i + 1)
        similarity_score = similarities[i]
        top_5_docs.append({'document_id': doc_id, 'score': similarity_score, 'text': get_original_comment(doc_id)})

    print("En Benzer 5 Belge:")
    for doc in top_5_docs:
        print(f"  Belge ID: {doc['document_id']}, Skor: {doc['score']:.4f}, Metin: '{doc['text']}'")

    # Jaccard benzerliği için en benzer 5 belge ID'sini saklıyorum
    all_model_top_docs[model_name] = [doc['document_id'] for doc in top_5_docs]

# Stemmed Word2Vec modelleri için işlem yapıyorum
for param in W2V_PARAMS:
    model_type = param['model_type']
    vector_size = param['vector_size']
    window = param['window']
    model_filename = f"stemmed_model_{model_type}_vs{vector_size}_w{window}.model"
    model_name = f"Stemmed Word2Vec ({model_type}, VS={vector_size}, W={window})"

    try:
        model = Word2Vec.load(MODELS_DIR + model_filename)
    except FileNotFoundError:
        print(f"Uyarı: {model_filename} modeli bulunamadı. Atlanıyor.")
        continue

    print(f"\n{model_name}:")

    # Giriş metni için ortalama vektörü alıyorum
    input_avg_vec = get_avg_vector(input_tokens_stemmed_w2v, model)

    # Korpustaki tüm belgelerle benzerlikleri hesaplıyorum
    document_vectors = []
    for doc_token_lists in df_stemmed_processed['comments_stemmed']:
        flat_doc_tokens = [token for sublist in doc_token_lists for token in sublist]
        document_vectors.append(get_avg_vector(flat_doc_tokens, model))
    document_vectors_matrix = np.array(document_vectors)

    similarities = cosine_similarity(input_avg_vec.reshape(1, -1), document_vectors_matrix).flatten()

    # En benzer 5 belgeyi alıyorum
    top_5_indices = similarities.argsort()[-5:][::-1]
    top_5_docs = []
    for i in top_5_indices:
        doc_id = 'doc' + str(i + 1)
        similarity_score = similarities[i]
        top_5_docs.append({'document_id': doc_id, 'score': similarity_score, 'text': get_original_comment(doc_id)})

    print("En Benzer 5 Belge:")
    for doc in top_5_docs:
        print(f"  Belge ID: {doc['document_id']}, Skor: {doc['score']:.4f}, Metin: '{doc['text']}'")

    # Jaccard benzerliği için en benzer 5 belge ID'sini saklıyorum
    all_model_top_docs[model_name] = [doc['document_id'] for doc in top_5_docs]

# Jaccard hesaplaması için TF-IDF sonuçlarını all_model_top_docs'a ekliyorum
all_model_top_docs["TF-IDF (Lemmatized)"] = [doc['document_id'] for doc in top_5_docs_lemma_tfidf]
all_model_top_docs["TF-IDF (Stemmed)"] = [doc['document_id'] for doc in top_5_docs_stem_tfidf]

print("Word2Vec benzerlik hesaplamaları tamamlandı.")


--- Word2Vec Benzerlik Hesaplamaları ---

Lemmatized Word2Vec (cbow, VS=100, W=2):
En Benzer 5 Belge:
  Belge ID: doc89, Skor: 0.9911, Metin: 'fix it!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!'
  Belge ID: doc272, Skor: 0.9907, Metin: 'I bought a 2014 Ford Focus used on 10/05/2015. It is the biggest piece of crap I have ever owned. It was a rental car when purchased from Manly Honda in Santa Rosa. I would like to have the fixed it under the California Lemon Law.'
  Belge ID: doc361, Skor: 0.9907, Metin: 'I have noticed this problem on similar vehicles. Recalls were made, however I was never notified. It is becoming worse as time goes on.'
  Belge ID: doc418, Skor: 0.9907, Metin: '2017 Honda CRV IS SO POOR.'
  Belge ID: doc143, Skor: 0.9907, Metin: 'This is the second one on the same vehicle. Makes American made stuff not worth looking at. Thanks Cry Slur! You just worry yourself about making sure you can fart through silk shorts while you put out shoddy cheap breakable product. I wa

In [6]:
print("\n--- Jaccard Benzerlik Matrisi ---")

def jaccard_similarity(list1, list2):
    s1 = set(list1)
    s2 = set(list2)
    intersection = len(s1.intersection(s2))
    union = len(s1.union(s2))
    return intersection / union if union != 0 else 0

model_names = list(all_model_top_docs.keys())
jaccard_matrix = pd.DataFrame(0.0, index=model_names, columns=model_names)

for i in range(len(model_names)):
    for j in range(len(model_names)):
        model_a_name = model_names[i]
        model_b_name = model_names[j]
        
        # Örneğimle eşleşmek için: "Model Adı", "5 Benzer Metin"
        # ve "Model tfidf_lemmatized ilk 5 sonucu: A={doc1, doc2, doc3, doc4, doc5}"
        # Jaccard hesaplaması için belgelerin yalnızca belge ID'leri olduğundan emin oluyorum.
        list_a = all_model_top_docs[model_a_name]
        list_b = all_model_top_docs[model_b_name]
        
        jaccard_matrix.loc[model_a_name, model_b_name] = jaccard_similarity(list_a, list_b)

# Jaccard Matrisini yazdırıyorum (daha iyi okunabilirlik için biçimlendirilmiş)
print(jaccard_matrix.round(2))

print("\nJaccard benzerlik matrisi oluşturuldu ve yazdırıldı.")


--- Jaccard Benzerlik Matrisi ---
                                             Lemmatized Word2Vec (cbow, VS=100, W=2)  \
Lemmatized Word2Vec (cbow, VS=100, W=2)                                         1.00   
Lemmatized Word2Vec (skipgram, VS=100, W=2)                                     0.11   
Lemmatized Word2Vec (cbow, VS=100, W=4)                                         0.43   
Lemmatized Word2Vec (skipgram, VS=100, W=4)                                     0.00   
Lemmatized Word2Vec (cbow, VS=300, W=2)                                         0.00   
Lemmatized Word2Vec (skipgram, VS=300, W=2)                                     0.11   
Lemmatized Word2Vec (cbow, VS=300, W=4)                                         0.11   
Lemmatized Word2Vec (skipgram, VS=300, W=4)                                     0.00   
Stemmed Word2Vec (cbow, VS=100, W=2)                                            0.00   
Stemmed Word2Vec (skipgram, VS=100, W=2)                                        0.00 